# Making a mock observation from the EAGLE simulations

This script is an example script to lay out the individual steps of making mock observations.

The steps are as follows:

1. Load the EAGLE simulated data.  Specify the distance at which we want to observe the cosmic web i.e. redshift of central wavelength of filter and the resolution that the data should be binned to.  The minimum resolution at 50Mpc away is about 14".
2. Cut out a chunk of the simulated data that corresponds to the Dragonfly FOV.  (switched with step 3 because takes too long for the whole simulation)
3. Add the noise to the simulated data to mimic observational noises (includes sky background noise, read out noise, dark current noise, and shot noise).  This depends on the exposure time of the mock observation.
4. Plot the data to check and see what it looks like.

Ta da!

In [1]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import os
import get_halpha_SB

%matplotlib inline

%run 'load_data.ipynb'
%run 'add_noise.ipynb'
%run 'extract_FOV.ipynb'

In [2]:
machine='coho'
resolution = 14.; slicewidth = 20; exptime = 1000.*60.*60.; CMOS = False; R_squared= None

for distance in ['500Mpc','200Mpc','100Mpc','50Mpc']:
    print "****** Working on mock observation for distance %s ******"%distance
    print 'loading the data...'
    data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance,\
                          slicewidth=slicewidth,CMOS=CMOS,R_squared=R_squared)
    print 'extracting the FOV...'
    data_FOV, xystarts, size = extractFOV(data_tuple,resolution,distance)
    
    print 'adding noise to the data...'
    noiseadded_signal,B_sky_array,R_array = addnoise(data_FOV,resolution,exptime=exptime,debugging=True)
    
    print 'saving the noise-added data...'
    if CMOS:
        cams = 'newcam'
    else:
        cams = 'oldcam'
    if R_squared:
        cams = cams+'_Rsquared%s'%R_squared

    fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth)
    R_fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_RARRAY.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth)
    B_sky_fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_BSKYARRAY.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth)

    print "Saving mock observation as %s"%fname
    np.savez(fname,noiseadded_signal)
    print "Saving readout noise array as %s"%R_fname
    np.savez(R_fname,R_array)
    print "Saving the sky background array as %s"%B_sky_fname
    np.savez(B_sky_fname,B_sky_array)
    
    print "****** Done mock observation for distance %s ******"%distance

Loading data of slicewidth 20 (after rounding), number of slices is 4
data_50Mpc_14arcsec_20slwd.npz exists (and is special because cant save as tuple), loading now...
